In [1]:
%load_ext autoreload
%autoreload 2
import numpy
import pandas
import io
import uuid
import psycopg.sql
import pyarrow
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.parquet
import jobqueue
from jobqueue.connection_manager import ConnectionManager



import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import datetime

from typing import Callable, List

from psycopg import sql

import dmp.keras_interface.model_serialization as model_serialization
from dmp.task.experiment.training_experiment.training_epoch import TrainingEpoch
from dmp.postgres_interface.element.column import Column
from dmp.postgres_interface.element.table import Table
from dmp.postgres_interface.element.column_group import ColumnGroup

from dmp.util.butter_e_export import *

pd.options.display.max_seq_items = None
credentials = jobqueue.load_credentials("dmp")

2023-11-28 14:44:53.379727: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-28 14:44:53.549076: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-28 14:44:53.549098: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-28 14:44:53.549980: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-28 14:44:53.632516: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-28 14:44:53.633883: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
from psycopg import ClientCursor


print(f"run vars {vars(run)}")

columns = (
    run
    + ColumnGroup(*[c for c in job_status.columns if c.name != "id"])
    + job_data.command
)
print(columns.names)


def passthrough(row, index, value, column, data):
    data[column.name] = value


column_converters: List[Callable] = [passthrough for _ in columns]


def flatten_json(json_obj, destination=None, parent_key="", separator="_"):
    if isinstance(destination, dict):
        flattened = destination
    else:
        flattened = {}

    for key, value in json_obj.items():
        new_key = f"{parent_key}{separator}{key}" if parent_key else key
        if isinstance(value, dict):
            flattened.update(flatten_json(value, new_key, separator=separator))
        else:
            flattened[new_key] = value
    return flattened


column_converters[
    columns.get_index_of(job_data.command)
] = lambda row, index, value, column, data: flatten_json(value, destination=data)
column_converters[
    columns.get_index_of(run.run_data)
] = lambda row, index, value, column, data: flatten_json(value, destination=data)


def parquet_to_dataframe(row, index, value, column, data):
    with io.BytesIO(value) as buffer:
        data[column.name] = (
            pyarrow.parquet.read_table(pyarrow.PythonFile(buffer, mode="r"))
            .to_pandas()
            .sort_values(by="epoch")
        )


column_converters[columns.get_index_of(run.run_history)] = parquet_to_dataframe
column_converters[columns.get_index_of(run.run_extended_history)] = parquet_to_dataframe


dfs = []


run vars {'_name': 'run', '_columns': (), '_index': None}
('experiment_id', 'run_timestamp', 'run_id', 'job_id', 'seed', 'slurm_job_id', 'task_version', 'num_nodes', 'num_cpus', 'num_gpus', 'gpu_memory', 'host_name', 'batch', 'run_data', 'run_history', 'run_extended_history', 'queue', 'status', 'priority', 'start_time', 'update_time', 'worker', 'error_count', 'error', 'parent', 'command')
{Column(_name='experiment_id', type_name='uuid'): 0, Column(_name='run_timestamp', type_name='timestamp'): 1, Column(_name='run_id', type_name='uuid'): 2, Column(_name='job_id', type_name='uuid'): 3, Column(_name='seed', type_name='bigint'): 4, Column(_name='slurm_job_id', type_name='bigint'): 5, Column(_name='task_version', type_name='smallint'): 6, Column(_name='num_nodes', type_name='smallint'): 7, Column(_name='num_cpus', type_name='smallint'): 8, Column(_name='num_gpus', type_name='smallint'): 9, Column(_name='gpu_memory', type_name='integer'): 10, Column(_name='host_name', type_name='text'): 11,

In [15]:

def get_data(query_search):
    with ConnectionManager(credentials) as connection:
        query = psycopg.sql.SQL(
            """
    SELECT
        {columns}
    FROM
        {run},
        {job_status},
        {job_data}
    WHERE
        {run}.batch LIKE {pattern}
        AND {job_status}.id = {run}.run_id
        AND {job_status}.id = {job_data}.id
        AND {job_status}.status = 2
        AND {job_data}.command @> {json_data}::jsonb
    LIMIT 2
    """
        ).format(
            columns=columns.columns_sql,
            run=run.identifier,
            job_status=job_status.identifier,
            job_data=job_data.identifier,
            pattern=sql.Literal("%energy%"),
            json_data=sql.Literal(query_search),
        )

        # with ClientCursor(connection) as c:
        #     print(c.mogrify(query))

        with connection.cursor(binary=True) as cursor:
            cursor.execute(query, binary=True)

            for row in cursor:
                row_data = {}
                for i, (column, column_converter) in enumerate(
                    zip(columns, column_converters)
                ):
                    column_converter(row, i, row[i], column, row_data)

                row_df = row_data["run_history"]
                row_df = row_df.join(
                    row_data["run_extended_history"], on="epoch", how="left", rsuffix="_"
                )
                print(list(row_df.columns.values))
                for k in ("run_history", "run_extended_history"):
                    del row_data[k]

                for k, v in row_data.items():
                    if k in row_df:
                        pass
                    if isinstance(v, list):
                        row_df[k] = [v] * len(row_df)
                    else:
                        row_df[k] = v
                    # data.setdefault(k, []).append(v)
                dfs.append(row_df)

    data = pandas.concat(dfs)

    for index, row in data.iterrows():
        for column in data.columns:
            if isinstance(row[column], uuid.UUID):
                data.at[index, column] = str(row[column])

    return data



In [16]:
get_data("{}")

['test_accuracy', 'train_accuracy', 'test_loss', 'train_loss', 'epoch', 'test_cosine_similarity', 'train_cosine_similarity', 'test_kullback_leibler_divergence', 'train_kullback_leibler_divergence', 'epoch_']
['test_accuracy', 'train_accuracy', 'test_loss', 'train_loss', 'epoch', 'test_cosine_similarity', 'train_cosine_similarity', 'test_kullback_leibler_divergence', 'train_kullback_leibler_divergence', 'epoch_']


/home/ctripp/dmp/env0/lib/python3.10/site-packages/pandas/io/formats/format.py:1595: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/home/ctripp/dmp/env0/lib/python3.10/site-packages/pandas/io/formats/format.py:1596: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/home/ctripp/dmp/env0/lib/python3.10/site-packages/pandas/io/formats/format.py:1596: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/home/ctripp/dmp/env0/lib/python3.10/site-packages/pandas/io/formats/format.py:1595: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/home/ctripp/dmp/env0/lib/python3.10/site-packages/pandas/io/formats/format.py:1596: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals 

,test_accuracy,train_accuracy,test_loss,train_loss,epoch,test_cosine_similarity,train_cosine_similarity,test_kullback_leibler_divergence,train_kullback_leibler_divergence,epoch_,...,name,method,source,test_split,label_noise,validation_split,run_tags,class,learning_rate,early_stopping
0,0.077000,0.034250,2.384217,2.399859,1,0.315702,0.309997,2.352188,2.370012,2.0,...,201_pol,shuffled_train_test_split,pmlb,0.2,0.0,0.0,None,Adam,0.0001,None
1,0.361667,0.197000,2.352203,2.370025,2,0.328296,0.320781,2.314635,2.336985,3.0,...,201_pol,shuffled_train_test_split,pmlb,0.2,0.0,0.0,None,Adam,0.0001,None
2,0.586333,0.468333,2.314649,2.336999,3,0.345632,0.334882,2.266197,2.296302,4.0,...,201_pol,shuffled_train_test_split,pmlb,0.2,0.0,0.0,None,Adam,0.0001,None
3,0.635333,0.595083,2.266211,2.296317,4,0.370784,0.354888,2.201205,2.242655,5.0,...,201_pol,shuffled_train_test_split,pmlb,0.2,0.0,0.0,None,Adam,0.0001,None
4,0.637667,0.618833,2.201220,2.242668,5,0.407779,0.384186,2.113321,2.170389,6.0,...,201_pol,shuffled_train_test_split,pmlb,0.2,0.0,0.0,None,Adam,0.0001,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.884571,0.902000,0.463969,0.352319,2996,0.898308,0.914486,0.448271,0.350147,2997.0,...,mnist,shuffled_train_test_split,pmlb,0.2,0.0,0.0,None,Adam,0.0001,None
2996,0.884571,0.902036,0.463970,0.352322,2997,0.898307,0.914497,0.448339,0.350109,2998.0,...,mnist,shuffled_train_test_split,pmlb,0.2,0.0,0.0,None,Adam,0.0001,None
2997,0.884714,0.902000,0.464071,0.352285,2998,0.898291,0.914491,0.448355,0.350124,2999.0,...,mnist,shuffled_train_test_split,pmlb,0.2,0.0,0.0,None,Adam,0.0001,None
2998,0.884643,0.902125,0.464062,0.352299,2999,0.898301,0.914496,0.448314,0.350109,3000.0,...,mnist,shuffled_train_test_split,pmlb,0.2,0.0,0.0,None,Adam,0.0001,None


In [4]:

import tqdm as tqmd
def save_data(queries:list):
    for query in tqmd.tqdm(queries):
        data = get_data(query)
        # convert to pyarrow table
        table = pa.Table.from_pandas(data)

        # write to distributed parquet file saved as ['name','depth','size','shape']
        pq.write_to_dataset(table, root_path='/projects/gcomp/jgafur/dataset/', partition_cols=['name','shape','depth','size'])
        del data


In [5]:
import tqdm as tqmd
from multiprocessing import Pool
import pyarrow as pa
import pyarrow.parquet as pq


def save_parallel(query):
    data = get_data(query)
    table = pa.Table.from_pandas(data)
    pq.write_to_dataset(table, root_path='/projects/gcomp/jgafur/dataset/', partition_cols=['name', 'shape', 'depth', 'size'])
    del data

def save_data_parallelized(queries):
    with Pool() as pool:
        list(tqmd.tqdm(pool.imap(save_parallel, queries), total=len(queries)))


In [6]:
import json
def get_json(**kwargs):
    # Given dictionary
    given_dict = {}

    # Update the given dictionary with user input
    given_dict.update(kwargs)

    # Convert the updated dictionary to JSON
    json_result = json.dumps(given_dict, indent=2)

    return json_result

In [7]:
query_search1 = get_json( dataset={"name": "sleep"}
        )

query_search2 = get_json(dataset={"name":"banana"},
        )

query_search3 = get_json(dataset={"name":"connect_4"},
        )

query_search4 = get_json(dataset={"name":"mnist"},
        )

query_search5 = get_json(dataset={"name":"nursery"},
        )

query_search6 = get_json(dataset={"name":"splice"},
        )

query_search7 = get_json(dataset={"name":"wine_quality_white"},
        )

query_search8 = get_json(dataset={"name":"201_pol"},
        )

query_search9 = get_json(dataset={"name":"294_satellite_image"},
        )

query_search10 = get_json(dataset={"name":"505_tecator"},
        )

query_search11 = get_json(dataset={"name":"529_pollen"},
        )

query_search12 = get_json(dataset={"name":"537_houses"},
        )

query_search13 = get_json(dataset={"name":"adult"},
        )


save_data_parallelized([query_search1, query_search2, query_search3, query_search4, query_search5, query_search6, query_search7, query_search8, query_search9, query_search10, query_search11, query_search12, query_search13])

  0%|          | 0/13 [00:00<?, ?it/s]


    SELECT
        "experiment_id","run_timestamp","run_id","job_id","seed","slurm_job_id","task_version","num_nodes","num_cpus","num_gpus","gpu_memory","host_name","batch","run_data","run_history","run_extended_history","queue","status","priority","start_time","update_time","worker","error_count","error","parent","command"
    FROM
        "run",
        "job_status",
        "job_data"
    WHERE
        "run".batch LIKE '%energy%'
        AND "job_status".id = "run".run_id
        AND "job_status".id = "job_data".id
        AND "job_status".status = 2
        AND "job_data".command @> '{
  "dataset": {
    "name": "splice"
  }
}'::jsonb
    ORDER BY
        experiment_id, run_id;
    
    SELECT
        "experiment_id","run_timestamp","run_id","job_id","seed","slurm_job_id","task_version","num_nodes","num_cpus","num_gpus","gpu_memory","host_name","batch","run_data","run_history","run_extended_history","queue","status","priority","start_time","update_time","worker","error_count","err

  0%|          | 0/13 [02:56<?, ?it/s]


KeyboardInterrupt: 